In [1]:
# !pip install allennlp

     |████████████████████████████████| 625 kB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 60.4 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 58.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 53.7 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 59.6 MB/s eta 0:00:01
  Using cached tqdm-4.60.0-py2.py3-none-any.whl (75 kB)
     |████████████████████████████████| 259 kB 54.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 61.6 MB/s eta 0:00:01
     |████████████████████████████████| 12.9 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 59.4 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 48.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 456 kB 60.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 57.5 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 

     |████████████████████████████████| 3.3 MB 47.5 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 64.1 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 63.1 MB/s eta 0:00:01
  Using cached GitPython-3.1.14-py3-none-any.whl (159 kB)
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10175 sha256=f98715cafc0aad246ea9eaaab4f5814750405a78b5e8b2ae2739b05d0fbca283
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e6/3b/34/ae59fc8d35c37f01099425ab73599e45e9b9b599a7ccc2c45f
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp36-cp36m-linux_x86_64.whl size=377789 sha256=01d4c2655914db839723466df731818f2c5b50e6992a2ac6e5c4598d8eda4209
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7e/ad/c9/

In [2]:
from pathlib import Path
from tqdm import tqdm
import pickle

In [50]:
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [4]:
import pandas as pd

In [5]:
data = Path('data_prep/final_data/en/')
elmo_path = Path('models/elmo_small/')

In [6]:
raw = pd.read_csv('data_prep/data/entries_raw.csv')
train = pd.read_csv(data / 'sentences_en_train.csv')
test = pd.read_csv(data / 'sentences_en_test.csv')

In [7]:
sentences = [x.split() for x in train.sentence_text]

In [8]:
base = Path('/home/ec2-user/SageMaker/experiments-dfs/models/elmo_small/results/sentence_embedding')
sentence_embedding = pd.read_pickle(base / f'sentence_output.pickle')

In [32]:
def process_train(df, sector):
    relevant_train = df[df.is_relevant == 1]
    relevant_train.sector_ids = relevant_train.sector_ids.apply(eval)
    relevant_train = relevant_train[relevant_train.sector_ids.apply(len) > 0]
    
    positive_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector in x)]
    negative_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector not in x)]

    positive_train.sector_ids = 1
    negative_train.sector_ids = 0
    train_df = pd.concat([positive_train, negative_train])
    train_df = train_df.sample(frac=1)#.reset_index(drop=True)
    
    return train_df

In [53]:
train_df = process_train(train, 4)
test_df = process_train(test, 4)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [60]:
X = sentence_embedding[train_df.index].numpy()
X_test = sentence_embedding[test_df.index].numpy()
y = train_df.sector_ids.values
y_test = test_df.sector_ids.values

In [61]:
logistic = LogisticRegression()

In [62]:
logistic.fit(X, y)

LogisticRegression()

In [63]:
preds = logistic.predict(X_test)

In [65]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.56      0.89      0.69      1646
           1       0.41      0.10      0.16      1260

    accuracy                           0.55      2906
   macro avg       0.49      0.50      0.43      2906
weighted avg       0.50      0.55      0.46      2906



In [8]:
model = Elmo(
    options_file=elmo_path / 'elmo_2x1024_128_2048cnn_1xhighway_options.json',
    weight_file=elmo_path / 'elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5',
    num_output_representations=1
)
model.cuda()

Elmo(
  (_elmo_lstm): _ElmoBiLm(
    (_token_embedder): _ElmoCharacterEncoder(
      (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
      (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
      (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
      (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
      (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
      (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
      (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
      (_highways): Highway(
        (_layers): ModuleList(
          (0): Linear(in_features=2048, out_features=4096, bias=True)
        )
      )
      (_projection): Linear(in_features=2048, out_features=128, bias=True)
    )
    (_elmo_lstm): ElmoLstm(
      (forward_layer_0): LstmCellWithProjection(
        (input_linearity): Linear(in_features=128, out_features=4096, bias=False)
        (state_linearity): Linear(in_features=128, out_featu

In [9]:
output = []
batch_size = 64
with torch.no_grad():
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = batch_to_ids(sentences[i:i+batch_size])[:, :350, :].cuda()
#         print(batch.shape)
    #     print(batch.shape)
        output.append(model(batch)['elmo_representations'][0].cpu())
#         batch.cpu()

100%|██████████| 3000/3000 [53:13<00:00,  1.06s/it]  


In [10]:
len(output)

3000

In [11]:
# base = Path('/home/ec2-user/SageMaker/experiments-dfs/models/elmo_small/results/sentence_embedding')
# for i in tqdm(range(0, len(output), 64)):
#     with open(base / f'{i}.pickle', 'wb') as f:
#         pickle.dump(output[i:i+64], f)

In [12]:
output[0].shape

torch.Size([64, 84, 256])

In [13]:
sentence_output = [x.mean(axis=1) for x in output]
sentence_output = torch.cat(sentence_output)

In [14]:
with open(s, 'wb') as f:
    pickle.dump(sentence_output, f)

NameError: name 'base' is not defined

In [ ]:
!pwd